In [1]:
import sys
# sys.path.insert(0,'/eos/home-a/antoniov/SWAN_projects/env/uproot4/lib/python3.7/site-packages')
# sys.path.insert(0,'/eos/home-a/antoniov/SWAN_projects/env/uproot/lib/python3.8/site-packages')
sys.path.insert(0,'/eos/home-a/antoniov/SWAN_projects/env/uproot-py39/lib/python3.9/site-packages')
print ( sys.path )

import os
import uproot
import awkward as ak
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mplhep
import numba as nb
import h5py

print ( "uproot:", uproot.__version__ )

['/eos/home-a/antoniov/SWAN_projects/env/uproot-py39/lib/python3.9/site-packages', '/cvmfs/sft.cern.ch/lcg/releases/condor/8.9.11-1f759/x86_64-centos7-gcc8-opt/lib/python3', '/cvmfs/sft.cern.ch/lcg/views/LCG_101swan/x86_64-centos7-gcc8-opt/lib/python3.9/site-packages/itk', '/cvmfs/sft.cern.ch/lcg/views/LCG_101swan/x86_64-centos7-gcc8-opt/python', '/cvmfs/sft.cern.ch/lcg/views/LCG_101swan/x86_64-centos7-gcc8-opt/lib', '', '/cvmfs/sft.cern.ch/lcg/views/LCG_101swan/x86_64-centos7-gcc8-opt/lib/python3.9/site-packages', '/cvmfs/sft.cern.ch/lcg/releases/Python/3.9.6-b0f98/x86_64-centos7-gcc8-opt/lib/python39.zip', '/cvmfs/sft.cern.ch/lcg/releases/Python/3.9.6-b0f98/x86_64-centos7-gcc8-opt/lib/python3.9', '/cvmfs/sft.cern.ch/lcg/releases/Python/3.9.6-b0f98/x86_64-centos7-gcc8-opt/lib/python3.9/lib-dynload', '/eos/home-a/antoniov/SWAN_projects/env/uproot-py39/lib/python3.9/site-packages', '/cvmfs/sft.cern.ch/lcg/views/LCG_101swan/x86_64-centos7-gcc8-opt/lib/python3.9/site-packages/IPython/exte

In [2]:
from create_table import create_table

Welcome to JupyROOT 6.24/00
{'muon': {'2017B': 2.360904801, '2017C1': 5.313012839, '2017C2': 3.264135878, '2017D': 4.074723964, '2017E': 8.958810514, '2017F1': 1.708478656, '2017F2': 7.877903151, '2017F3': 3.632463163}, 'electron': {'2017B': 2.259685729466727, '2017C1': 5.070112518026598, '2017C2': 3.114906113929596, '2017D': 3.8868791892596004, '2017E': 8.568260128452684, '2017F1': 1.629432474022848, '2017F2': 7.513416205912684, '2017F3': 3.4643999898374793}}
Luminosity 2017 muon: 37.190432966
Luminosity 2017 electron: 35.507092348908216
{'muon': {'2018A': 12.1, '2018B1': 6.38, '2018B2': 0.4, '2018C': 6.5297, '2018D1': 19.88, '2018D2': 10.4157}, 'electron': {'2018A': 12.1, '2018B1': 6.38, '2018B2': 0.4, '2018C': 6.5297, '2018D1': 19.88, '2018D2': 10.4157}}
Luminosity 2018 muon: 55.7054
Luminosity 2018 electron: 55.7054


In [ ]:
help( create_table )

In [ ]:
# dir( ak )
# help( ak.highlevel.Array )
# help( ak.full_like )
# help( ak.zeros_like )
# help( ak.copy )
help( ak.concatenate )

In [ ]:
#os.listdir("/eos/user/a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017B-Dilepton/190621_214052/0000")
#os.listdir("/eos/user/a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017C-Dilepton/190621_214124/0000")
#os.listdir("/eos/user/a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017D-Dilepton/190621_214154/")
#os.listdir("/eos/user/a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017E-Dilepton/190621_214224/0000")
#os.listdir("/eos/user/a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017F-Dilepton/190621_214253/0000")
#os.listdir("/eos/user/a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017F-Dilepton/190621_214253/0001")
#os.listdir("/eos/user/a/antoniov/Workspace/analysis/data/PPS/GGToWW_bSM-A0W1e-6_13TeV-fpmc-herwig6-signal-proton-propagation-dilepton/200420_160422/0000")
#os.listdir("/eos/user/a/antoniov/Workspace/analysis/data/PPS/GGToWW_bSM-A0W2e-6_13TeV-fpmc-herwig6-signal-proton-propagation-dilepton/200420_160449/0000")
#os.listdir("/eos/user/a/antoniov/Workspace/analysis/data/PPSp,.nGGToWW_bSM-A0W5e-6_13TeV-fpmc-herwig6-signal-proton-propagation-dilepton/200420_160355/0000")
#os.listdir("/eos/user/a/antoniov/Workspace/analysis/data/PPS/GGToWWToJJMuNu_PtL-15_13TeV-fpmc-herwig6-signal-proton-propagation-dilepton/200420_160255/0000")
#os.listdir("/eos/user/a/antoniov/Workspace/analysis/data/PPS/GGToWWToJJENu_PtL-15_13TeV-fpmc-herwig6-signal-proton-propagation-dilepton/200420_160227/0000")
#os.listdir("/eos/user/a/antoniov/Workspace/analysis/data/PPS/GGToWW_bSM-ACW5e-6_13TeV-fpmc-herwig6-signal-proton-propagation-dilepton/200424_185019/0000")
#os.listdir("/eos/user/a/antoniov/Workspace/analysis/data/PPS/GGToWW_bSM-ACW8e-6_13TeV-fpmc-herwig6-signal-proton-propagation-dilepton/200424_185049/0000")
#os.listdir("/eos/user/a/antoniov/Workspace/analysis/data/PPS/GGToWW_bSM-ACW2e-5_13TeV-fpmc-herwig6-signal-proton-propagation-dilepton/200424_185121/0000")

In [3]:
# fileName_ = "/eos/home-a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017B-Dilepton/190621_214052/0000/SlimmedNtuple_merged_0.root"
# fileName_ = "/eos/home-a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017B-withDilepton/0000/SlimmedNtuple_merged_0.root"
# fileName_ = "/eos/home-a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017B-withDilepton/SlimmedNtuple_merged.root"
# fileName_ = "/eos/home-a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2018A_merged.root"
# fileName_ = "/eos/home-a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2018D_merged.root"
# fileName_ = "/eos/home-a/antoniov/Workspace/analysis/data/PPS/GGToWW_bSM-A0W1e-6_13TeV-fpmc-herwig6-signal-proton-propagation-dilepton/200420_160422/0000/SlimmedNtuple_merged_0.root"
fileName_ = "/eos/home-a/antoniov/Workspace/analysis/data/PPS/SingleElectron-Run2017B/SingleElectron-Run2017B_merged.root"

root_ = uproot.open( fileName_ )

# tree_path_ = "demo/SlimmedNtuple"
tree_path_ = "SlimmedNtuple"

print ( "Number of events in tree: {}".format( np.array( root_[ tree_path_ + "/event"] ).size ) )

print ( root_[ tree_path_ ].keys() )

Number of events in tree: 64116
['muon_pt', 'muon_eta', 'muon_phi', 'muon_px', 'muon_py', 'muon_pz', 'muon_e', 'muon_charge', 'muon_iso', 'muon_dxy', 'muon_dz', 'electron_pt', 'electron_eta', 'electron_phi', 'electron_dxy', 'electron_dz', 'electron_px', 'electron_py', 'electron_pz', 'electron_e', 'electron_charge', 'met', 'met_x', 'met_y', 'met_phi', 'pfcand_nextracks', 'pfcand_nextracks_noDRl', 'num_bjets_ak8', 'num_bjets_ak4', 'num_jets_ak4', 'recoMWhad', 'recoMWlep', 'recoMWW', 'recoRapidityWW', 'recoWWpt', 'genWWpt', 'dphiWW', 'WLeptonicPt', 'WLeptonicPhi', 'WLeptonicEta', 'jet_pt', 'jet_px', 'jet_py', 'jet_pz', 'jet_energy', 'jet_phi', 'jet_eta', 'jet_mass', 'jet_tau1', 'jet_tau2', 'jet_corrmass', 'jet_vertexz', 'jet_jer_res', 'jet_jer_sf', 'jet_jer_sfup', 'jet_jer_sfdown', 'hlt', 'gen_W_pt', 'gen_W_charge', 'gen_jet_pt', 'gen_jet_phi', 'gen_jet_eta', 'gen_jet_energy', 'nVertices', 'pileupWeight', 'mc_pu_trueinteractions', 'mcWeight', 'run', 'event', 'lumiblock', 'crossingAngle', 

In [4]:
tree_ = root_[ tree_path_ ]
 
keys_nonproton = ["run", "event", "lumiblock", "crossingAngle", "nVertices",
                  "num_bjets_ak8", "num_bjets_ak4", "num_jets_ak4",
                  "pfcand_nextracks", "pfcand_nextracks_noDRl",
                  "recoMWhad", "recoMWlep", "recoMWW", "recoRapidityWW", "dphiWW", "WLeptonicPt", "WLeptonicEta", "WLeptonicPhi",
                  "pileupWeight", "mc_pu_trueinteractions", "mcWeight" ]
keys_nonproton.extend( tree_.keys( filter_name="jet*") )
keys_nonproton.extend( tree_.keys( filter_name="muon*") )
keys_nonproton.extend( tree_.keys( filter_name="met*") )
keys_proton = tree_.keys( filter_name="proton*")
keys_ppstrack = tree_.keys( filter_name="pps_track*")
keys = []
keys.extend( keys_nonproton )
keys.extend( keys_proton )
keys.extend( keys_ppstrack )
print ( keys )

how_ = None
#how_= "zip"

print ( "\n" )
print ( root_[ tree_path_ + "/proton_xi" ] )
print ( np.array( root_[ tree_path_ + "/proton_xi" ] ) )
print ( tree_.arrays( "proton_xi", library="ak", how=how_ ) )

keys_extra = [ 'proton_trackx2', 'proton_tracky2', 'proton_trackpixshift2', 'proton_rpid2' ]
for key_ in keys_extra:
    if key_ in keys: keys.remove( key_ )
print ( "\n" )
print ( keys )

arrays = tree_.arrays( keys, library="ak", how=how_, entry_stop=10 )
print ( "\n" )
print ( arrays )
print ( ak.to_list( arrays ) )

arrays_extra = tree_.arrays( keys_extra, library="ak", how=how_, entry_stop=10 )
print ( "\n" )
print ( arrays_extra )
print ( ak.to_list( arrays_extra ) )


['run', 'event', 'lumiblock', 'crossingAngle', 'nVertices', 'num_bjets_ak8', 'num_bjets_ak4', 'num_jets_ak4', 'pfcand_nextracks', 'pfcand_nextracks_noDRl', 'recoMWhad', 'recoMWlep', 'recoMWW', 'recoRapidityWW', 'dphiWW', 'WLeptonicPt', 'WLeptonicEta', 'WLeptonicPhi', 'pileupWeight', 'mc_pu_trueinteractions', 'mcWeight', 'jet_pt', 'jet_px', 'jet_py', 'jet_pz', 'jet_energy', 'jet_phi', 'jet_eta', 'jet_mass', 'jet_tau1', 'jet_tau2', 'jet_corrmass', 'jet_vertexz', 'jet_jer_res', 'jet_jer_sf', 'jet_jer_sfup', 'jet_jer_sfdown', 'muon_pt', 'muon_eta', 'muon_phi', 'muon_px', 'muon_py', 'muon_pz', 'muon_e', 'muon_charge', 'muon_iso', 'muon_dxy', 'muon_dz', 'met', 'met_x', 'met_y', 'met_phi', 'proton_xi', 'proton_thx', 'proton_thy', 'proton_t', 'proton_ismultirp_', 'proton_rpid', 'proton_arm', 'proton_time', 'proton_trackx1', 'proton_tracky1', 'proton_trackx2', 'proton_tracky2', 'proton_trackpixshift1', 'proton_trackpixshift2', 'proton_rpid1', 'proton_rpid2', 'pps_track_x', 'pps_track_y', 'pps_t

In [ ]:
arr_run = tree_.arrays( "run" )
print ( ak.min( arr_run ) )
print ( ak.max( arr_run ) )

In [ ]:
arr_xangle = tree_.arrays( "crossingAngle" )
arr_xangle
print ( ak.min( arr_xangle ) )
print ( ak.max( arr_xangle ) )

In [ ]:
if how_ == "zip":
    protons = arrays["proton"]
    protons["run"] = arrays["run"]
    protons["event"] = arrays["event"]
    protons["lumiblock"] = arrays["lumiblock"]
    print ( protons )
    print ( ak.to_list(protons) )
    
    ppstracks = arrays["pps_track"]
    ppstracks["run"] = arrays["run"]
    ppstracks["event"] = arrays["event"]
    ppstracks["lumiblock"] = arrays["lumiblock"]
    print ("\n")
    print ( ppstracks )
    print ( ak.to_list(ppstracks) )
    
    protons_extra = arrays_extra[ "proton" ]
    protons_multiRP = protons[ protons.ismultirp == 1 ]
    protons_multiRP[ "trackx2" ] = protons_extra[ "trackx2" ]
    protons_multiRP[ "tracky2" ] = protons_extra[ "tracky2" ]
    protons_multiRP[ "trackpixshift2" ] = protons_extra[ "trackpixshift2" ]
    protons_multiRP[ "rpid2" ] = protons_extra[ "rpid2" ]
    print ("\n")
    print ( protons_multiRP )
    print ( ak.to_list( protons_multiRP ) )  
    
else:
    arrays_proton = {}
    
    keys_ = keys_proton.copy()
    for key_ in keys_extra:
        if key_ in keys_: keys_.remove( key_ )
    print ( keys_ )
    
    for key_ in keys_: arrays_proton[ key_[ len("proton_") : ] ] = arrays[ key_ ]
#     arrays_proton[ "random" ] = ak.full_like( arrays_proton[ "xi" ], 0 )
    arrays_proton[ "random" ] = ak.zeros_like( arrays_proton[ "arm" ] )
    print ( arrays_proton[ "random" ], ak.num( arrays_proton[ "random" ] ) )
    print ( arrays_proton )
    protons = ak.zip( arrays_proton )
    print ( protons, ak.num( protons ) )
    print ( ak.to_list( protons ) )
    
    arrays_proton_extra = {}
    
    for key_ in keys_extra: arrays_proton_extra[ key_[ len("proton_") : ] ] = arrays_extra[ key_ ]
    print ("\n")
    print ( arrays_proton_extra )
    protons_extra = ak.zip( arrays_proton_extra )
    print ( protons_extra )
    print ( ak.to_list( protons_extra ) )
    
    protons_multiRP = protons[ protons.ismultirp_ == 1 ]
    protons_multiRP[ "trackx2" ] = protons_extra[ "trackx2" ]
    protons_multiRP[ "tracky2" ] = protons_extra[ "tracky2" ]
    protons_multiRP[ "trackpixshift2" ] = protons_extra[ "trackpixshift2" ]
    protons_multiRP[ "rpid2" ] = protons_extra[ "rpid2" ]
    print ("\n")
    print ( protons_multiRP )
    print ( ak.to_list( protons_multiRP ) )  
        

In [ ]:
proton_files_ = [
    "/eos/home-a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017B-withDilepton/SlimmedNtuple_merged.root",
    "/eos/home-a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017C-withDilepton/SlimmedNtuple_merged.root",
    "/eos/home-a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017D-withDilepton/SlimmedNtuple_merged.root",
    "/eos/home-a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017E-withDilepton/SlimmedNtuple_merged.root",
    "/eos/home-a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017F-withDilepton/SlimmedNtuple_merged.root"
]

how_ = None

step_size_ = 100000

fill_proton_extra_ = True

protons_all_ = None
protons_extra_all_ = None
ppstracks_all_ = None
for file_ in proton_files_:
    print ( file_ ) 
    root_ = uproot.open( file_ )

    print ( "Number of events in tree: {}".format( np.array( root_[ tree_path_ + "/event" ] ).size ) )

    tree_ = root_[ tree_path_ ]

    keys_nonproton_ = [ "run", "event", "lumiblock" ]
    keys_proton_ = tree_.keys( filter_name="proton*")
    keys_ppstrack_ = tree_.keys( filter_name="pps_track*")
    keys_proton_extra_ = [ 'proton_trackx2', 'proton_tracky2', 'proton_trackpixshift2', 'proton_rpid2' ]
    keys_ = []
    keys_.extend( keys_nonproton_ )
    keys_.extend( keys_proton_ )
    keys_.extend( keys_ppstrack_ )
    if how_ == "zip":
        for key_ in keys_proton_extra_:
            if key_ in keys_: keys_.remove( key_ )
    print ( keys_ )
    
    for events_ in tree_.iterate( keys_ , library="ak", how=how_, step_size=step_size_ ):
        print ( events_, len( events_ ) )
        
        # Fetch protons
        protons_ = None
        protons_extra_ = None
        ppstracks_ = None
        if how_ == "zip":
            protons_ = events_["proton"]
            ppstracks_ = events_["pps_track"]
        elif how_ is None:
            keys_proton_ = keys_proton_.copy()
            for key_ in keys_proton_extra_:
                if key_ in keys_proton_: keys_proton_.remove( key_ )

            arrays_proton_ = {}
            for key_ in keys_proton_: arrays_proton_[ key_[ len("proton_") : ] ] = events_[ key_ ]
            arrays_proton_[ "random" ] = ak.ones_like( arrays_proton_[ "arm" ] )
            protons_ = ak.zip( arrays_proton_ )

            if fill_proton_extra_:
                arrays_proton_extra_ = {}
                for key_ in keys_proton_extra_: arrays_proton_extra_[ key_[ len("proton_") : ] ] = events_[ key_ ]
                protons_extra_ = ak.zip( arrays_proton_extra_ )

            arrays_ppstrack_ = {}
            for key_ in keys_ppstrack_: arrays_ppstrack_[ key_[ len("pps_track_") : ] ] = events_[ key_ ]
            ppstracks_ = ak.zip( arrays_ppstrack_ )        
        
        protons_[ "run" ] = events_[ "run" ]
        protons_[ "lumiblock" ] = events_[ "lumiblock" ]
        protons_[ "event" ] = events_[ "event" ]
        ppstracks_[ "run" ] = events_[ "run" ]
        ppstracks_[ "lumiblock" ] = events_[ "lumiblock" ]
        ppstracks_[ "event" ] = events_[ "event" ]
        
        print ( protons_, len( protons_ ), ak.num( protons_ ) )
        if fill_proton_extra_:
            print ( protons_extra_, len( protons_extra_ ), ak.num( protons_extra_ ) )
        print ( ppstracks_, len( ppstracks_ ), ak.num( ppstracks_ ) )
        
        if protons_all_ is None:
            protons_all_ = protons_
        else:
            protons_all_ = ak.concatenate( [ protons_all_, protons_ ], axis=0 )
        if fill_proton_extra_:    
            if protons_extra_all_ is None:
                protons_extra_all_ = protons_extra_
            else:
                protons_extra_all_ = ak.concatenate( [ protons_extra_all_, protons_extra_ ], axis=0 )
        if ppstracks_all_ is None:
            ppstracks_all_ = ppstracks_
        else:
            ppstracks_all_ = ak.concatenate( [ ppstracks_all_, ppstracks_ ], axis=0 )
    # end iterate
# end loop files

print ( "Collections concatenated:" )
print ( protons_all_, len( protons_all_ ), ak.num( protons_all_ ) )
if fill_proton_extra_:
    print ( protons_extra_all_, len( protons_extra_all_ ), ak.num( protons_extra_all_ ) )
print ( ppstracks_all_, len( ppstracks_all_ ), ak.num( ppstracks_all_ ) )

np.random.seed( 42 )

index_rnd_ = np.random.permutation( len( protons_all_ ) )

protons_all_rnd_ = protons_all_[ index_rnd_ ]
protons_extra_all_rnd_ = protons_extra_all_[ index_rnd_ ]
ppstracks_all_rnd_ = ppstracks_all_[ index_rnd_ ]

print ( "Collections randomized:" )
print ( protons_all_rnd_, len( protons_all_rnd_ ), ak.num( protons_all_rnd_ ) )
if fill_proton_extra_:
    print ( protons_extra_all_rnd_, len( protons_extra_all_rnd_ ), ak.num( protons_extra_all_rnd_ ) )
print ( ppstracks_all_rnd_, len( ppstracks_all_rnd_ ), ak.num( ppstracks_all_rnd_ ) )


In [ ]:
event_idx_ = 4
print ( ak.to_list( protons_all_[ event_idx_ ] ) )
print ( ak.to_list( protons_extra_all_[ event_idx_ ] ) )
print ( ak.to_list( ppstracks_all_[ event_idx_ ] ) )

In [ ]:
# def create_table( fileNames, label, random_protons=False, resample_factor=-1, read_size="150MB", firstEvent=None, entryStop=None, debug=False ):

#     fileNames_ = fileNames
#     label_ = label
#     random_protons_ = random_protons
#     resample_factor_ = resample_factor
#     read_size_ = read_size
#     firstEvent_ = firstEvent
#     entryStop_ = entryStop

#     #how_ = None
#     how_ = "zip"
    
#     print ( "Random protons: {}".format( random_protons_ ) )
    
#     resample = False
#     if resample_factor_ > 1: resample = True
#     print ( "Resample: {} / Resample factor: {}".format( resample,  resample_factor_ ) )

#     np.random.seed( 42 )

#     dset_chunk_size = 50000

#     columns_protons = ( "run", "lumiblock", "event", "slice", "xi", "thx", "thy", "t", "ismultirp", "rpid", "arm",
#                         "jet0_pt", "jet0_eta", "jet0_phi", "jet0_energy", "jet0_mass", "jet0_corrmass", "jet0_tau1", "jet0_tau2", "jet0_vertexz",
#                         "muon0_pt", "muon0_eta", "muon0_phi", "muon0_energy", "muon0_charge", "muon0_iso", "muon0_dxy", "muon0_dz",
#                         "met", "met_x", "met_y", "met_phi",
#                         "nVertices",
#                         "num_bjets_ak8", "num_bjets_ak4", "num_jets_ak4",
#                         "pfcand_nextracks", "pfcand_nextracks_noDRl",
#                         "recoMWhad", "recoMWlep", "recoMWW", "recoRapidityWW", "dphiWW", "WLeptonicPt", "WLeptonicPhi" )

#     columns_ppstracks = ( "run", "lumiblock", "event", "slice", "x", "y", "rpid" ) 

#     protons_keys = {}
#     for col_ in columns_protons:
#         protons_keys[ col_ ] = col_

#     ppstracks_keys = {}
#     for col_ in columns_ppstracks:
#         ppstracks_keys[ col_ ] = col_

#     counts_label_protons_ = "Proton" if not random_protons_ else "ProtonRnd"

#     with h5py.File( 'output-' + label_ + '.h5', 'w') as f:

#         dset_protons_multiRP = f.create_dataset( 'protons_multiRP', ( dset_chunk_size, len( columns_protons ) ), compression="gzip", chunks=True, maxshape=( None , len( columns_protons ) ) )
#         print ( "Initial dataset shape: {}".format( dset_protons_multiRP.shape ) )

#         dset_protons_singleRP = f.create_dataset( 'protons_singleRP', ( dset_chunk_size, len( columns_protons ) ), compression="gzip", chunks=True, maxshape=( None , len( columns_protons ) ) )
#         print ( "Initial dataset shape: {}".format( dset_protons_singleRP.shape ) )

#         dset_ppstracks = f.create_dataset( 'ppstracks', ( dset_chunk_size, len( columns_ppstracks ) ), compression="gzip", chunks=True, maxshape=( None , len( columns_ppstracks ) ) )
#         print ( "Initial dataset shape: {}".format( dset_ppstracks.shape ) )

#         protons_multiRP_list = {}
#         for col_ in columns_protons:
#             protons_multiRP_list[ col_ ] = []           

#         protons_singleRP_list = {}
#         for col_ in columns_protons:
#             protons_singleRP_list[ col_ ] = []           

#         ppstracks_list = {}
#         for col_ in columns_ppstracks:
#             ppstracks_list[ col_ ] = []           

#         selections = None
#         counts = None

#         dset_multiRP_slice = 0
#         dset_multiRP_idx = 0
#         dset_multiRP_entries = 0

#         dset_singleRP_slice = 0
#         dset_singleRP_idx = 0
#         dset_singleRP_entries = 0

#         dset_ppstracks_slice = 0
#         dset_ppstracks_idx = 0
#         dset_ppstracks_entries = 0

#         for file_ in fileNames_:
#             print ( file_ ) 
#             root_ = uproot4.open( file_ )

#             print ( "Number of events in tree: {}".format( np.array( root_["demo/SlimmedNtuple/event"] ).size ) )

#             tree_ = root_["demo/SlimmedNtuple"]

#             keys_nonproton = [ "run", "event", "lumiblock", "nVertices",
#                                "num_bjets_ak8", "num_bjets_ak4", "num_jets_ak4",
#                                "pfcand_nextracks", "pfcand_nextracks_noDRl",
#                                "recoMWhad", "recoMWlep", "recoMWW", "recoRapidityWW", "dphiWW", "WLeptonicPt", "WLeptonicPhi" ]
#             keys_nonproton.extend( tree_.keys( filter_name="jet*") )
#             keys_nonproton.extend( tree_.keys( filter_name="muon*") )
#             keys_nonproton.extend( tree_.keys( filter_name="met*") )
#             keys = []
#             keys.extend( keys_nonproton )
#             keys.extend( tree_.keys( filter_name="proton*") )
#             keys.extend( tree_.keys( filter_name="pps*") )
#             keys_remove = [ 'proton_trackx2', 'proton_tracky2', 'proton_trackpixshift2', 'proton_rpid2' ]
#             for key_ in keys_remove:
#                 if key_ in keys: keys.remove( key_ )
#             print ( keys )

#             for events_ in tree_.iterate( keys , library="ak", how=how_, step_size=read_size_, entry_start=firstEvent_, entry_stop=entryStop_ ):
#                 print ( len(events_), events_ )
#                 print ( "Num jets: {}".format( ak.num( events_["jet"] ) ) )
#                 print ( "Num muons: {}".format( ak.num( events_["muon"] ) ) )
#                 print ( "Num protons: {}".format( ak.num( events_["proton"] ) ) )
#                 print ( "Num pps tracks: {}".format( ak.num( events_["pps_track"] ) ) )

#                 selections_ = []
#                 counts_ = []

#                 selections_.append( "All" )
#                 counts_.append( len( events_ ) )

#                 # Event selections
#                 msk_1jet = ( ak.num( events_["jet"] ) >= 1 )
#                 selections_.append( "Jet" )
#                 counts_.append( np.sum( np.array( msk_1jet ).astype("int32") ) )

#                 msk_1muon = msk_1jet & ( ak.num( events_["muon"] ) >= 1 )
#                 selections_.append( "Muon" )
#                 counts_.append( np.sum( np.array( msk_1muon ).astype("int32") ) )

#                 events_ = events_[ msk_1muon ]    

#                 selections_ = np.array( selections_ )
#                 counts_ = np.array( counts_ )

#                 # Repeat events by resample factor
#                 if resample:
#                     counts_ = counts_ * resample_factor_

#                 if selections is None:
#                     selections = selections_
#                     counts = counts_
#                 else:
#                     msk_selections = np.full_like( selections, False, dtype='bool' )
#                     for key in selections_:
#                         msk_selections |= ( selections == key )
#                     counts[ msk_selections ] += counts_

#                 # Repeat events by resample factor
#                 slices_ = np.zeros( len( events_ ), dtype=np.int32 )
#                 if resample:
#                     events_size_ = len( events_ )
#                     events_ = ak.concatenate( ( [events_] * resample_factor_ ), axis=0 )
#                     slices_ = np.zeros( resample_factor_ * events_size_, dtype=np.int32 )
#                     for idx_ in range( resample_factor_ ):
#                         slices_[ ( idx_ * events_size_ ) : ( ( idx_ + 1 ) * events_size_ ) ] = idx_

#                 events_[ "slice" ] = slices_

#                 # Randomize proton arrays
#                 #protons_ = events_["proton"]
#                 protons_ = None
#                 ppstracks_ = None
#                 if random_protons_:
#                     protons_ = events_["proton"]
#                     ppstracks_ = events_["pps_track"]

#                     index_rnd_ = np.random.permutation( len( events_ ) )

#                     protons_rnd_ = protons_[ index_rnd_ ]
#                     ppstracks_rnd_ = ppstracks_[ index_rnd_ ]

#                     events_[ "proton_rnd" ] = protons_rnd_
#                     events_[ "pps_track_rnd" ] = ppstracks_rnd_

#                     print ( "Num protons: {}".format( ak.num( events_["proton"] ) ) )
#                     print ( "Num protons randomized: {}".format( ak.num( events_["proton_rnd"] ) ) )
#                     print ( "Num pps tracks: {}".format( ak.num( events_["pps_track"] ) ) )
#                     print ( "Num pps tracks randomized: {}".format( ak.num( events_["pps_track_rnd"] ) ) )

#                 if not random_protons_:
#                     protons_ = events_["proton"]
#                     ppstracks_ = events_["pps_track"]
#                 else:
#                     protons_ = events_["proton_rnd"]
#                     ppstracks_ = events_["pps_track_rnd"]
                    
#                 print ( "Num protons: {}".format( ak.num( protons_ ) ) )
#                 print ( "Num pps tracks: {}".format( ak.num( ppstracks_ ) ) )

#                 protons_["run"]                    = events_["run"]
#                 protons_["lumiblock"]              = events_["lumiblock"]
#                 protons_["event"]                  = events_["event"]
#                 protons_["slice"]                  = events_["slice"]
#                 protons_["jet0_pt"]                = events_.jet.pt[:,0]
#                 protons_["jet0_eta"]               = events_.jet.eta[:,0]
#                 protons_["jet0_phi"]               = events_.jet.phi[:,0]
#                 protons_["jet0_energy"]            = events_.jet.energy[:,0]
#                 protons_["jet0_mass"]              = events_.jet.mass[:,0]
#                 protons_["jet0_corrmass"]          = events_.jet.corrmass[:,0]
#                 protons_["jet0_tau1"]              = events_.jet.tau1[:,0]
#                 protons_["jet0_tau2"]              = events_.jet.tau2[:,0]
#                 protons_["jet0_vertexz"]           = events_.jet.vertexz[:,0]
#                 protons_["muon0_pt"]               = events_.muon.pt[:,0]
#                 protons_["muon0_eta"]              = events_.muon.eta[:,0]
#                 protons_["muon0_phi"]              = events_.muon.phi[:,0]
#                 protons_["muon0_energy"]           = events_.muon.e[:,0]
#                 protons_["muon0_charge"]           = events_.muon.charge[:,0]
#                 protons_["muon0_iso"]              = events_.muon.iso[:,0]
#                 protons_["muon0_dxy"]              = events_.muon.dxy[:,0]
#                 protons_["muon0_dz"]               = events_.muon.dz[:,0]
#                 protons_["met"]                    = events_["met"]
#                 protons_["met_x"]                  = events_["met_x"]
#                 protons_["met_y"]                  = events_["met_y"]
#                 protons_["met_phi"]                = events_["met_phi"]
#                 protons_["nVertices"]              = events_["nVertices"]
#                 protons_["num_bjets_ak8"]          = events_["num_bjets_ak8"]
#                 protons_["num_bjets_ak4"]          = events_["num_bjets_ak4"]
#                 protons_["num_jets_ak4"]           = events_["num_jets_ak4"]
#                 protons_["pfcand_nextracks"]       = events_["pfcand_nextracks"]
#                 protons_["pfcand_nextracks_noDRl"] = events_["pfcand_nextracks_noDRl"]
#                 protons_["recoMWhad"]              = events_["recoMWhad"]
#                 protons_["recoMWlep"]              = events_["recoMWlep"]
#                 protons_["recoMWW"]                = events_["recoMWW"]
#                 protons_["recoRapidityWW"]         = events_["recoRapidityWW"]
#                 protons_["dphiWW"]                 = events_["dphiWW"]
#                 protons_["WLeptonicPt"]            = events_["WLeptonicPt"]
#                 protons_["WLeptonicPhi"]           = events_["WLeptonicPhi"]
#                 #protons_["x1"] = -999.
#                 #protons_["y1"] = -999.
#                 #protons_["x2"] = -999.
#                 #protons_["y2"] = -999.
                
#                 #ppstracks_ = events_["pps_track"]
#                 ppstracks_["run"] = events_["run"]
#                 ppstracks_["lumiblock"] = events_["lumiblock"]
#                 ppstracks_["event"] = events_["event"]
#                 ppstracks_["slice"] = events_["slice"]

#                 protons_singleRP_ = protons_[ protons_.ismultirp == 0 ]
#                 protons_multiRP_ = protons_[ protons_.ismultirp == 1 ]

#                 protons_singleRP_byRP_ = {}
#                 ppstracks_byRP_ = {}
#                 protons_multiRP_byArm_ = {}
#                 for rpid in ( 3, 23, 103, 123 ):
#                     #arm = -1
#                     #if   rpid == 3   or rpid == 23 : arm = 0
#                     #elif rpid == 103 or rpid == 123 : arm = 1
#                     #print ( "Arm: {}".format( arm ) )

#                     protons_singleRP_byRP_[ rpid ] =  protons_singleRP_[ protons_singleRP_.rpid == rpid ]
#                     ppstracks_byRP_[ rpid ] = ppstracks_[ ppstracks_.rpid == rpid ]
#                     #protons_singleRP_byRP_[ rpid ]["x1"] = ppstracks_byRP_[ rpid ].x
#                     #protons_singleRP_byRP_[ rpid ]["y1"] = ppstracks_byRP_[ rpid ].y

#                     print ( "\nNum protons RP {}: {}".format( rpid, ak.num( protons_singleRP_byRP_[ rpid ] ) ) )
#                     if debug:
#                         print ( ak.to_list( protons_singleRP_byRP_[ rpid ] ) )
#                         print ("\n")
#                         print ( ak.to_list( ppstracks_byRP_[ rpid ] ) )

#                 for arm in ( 0, 1 ):
#                     protons_multiRP_byArm_[ arm ] = protons_multiRP_[ protons_multiRP_.arm == arm ]

#                     print ( "\nNum multi-RP protons Arm {}: {}".format( arm, ak.num( protons_multiRP_byArm_[ arm ] ) ) )
#                     if debug:
#                         print ( ak.to_list( protons_multiRP_byArm_[ arm ] ) )

#                 #msk  =  np.array( ak.num( protons_singleRP_byRP_[ 3 ].xi ) == 1 )
#                 #msk &= np.array( ak.num( protons_singleRP_byRP_[ 23 ].xi ) == 1 )
#                 #msk &= np.array( ak.num( protons_singleRP_byRP_[ 103 ].xi ) == 1 )
#                 #msk &= np.array( ak.num( protons_singleRP_byRP_[ 123 ].xi ) == 1 )    

#                 msk_protons  = np.array( ak.num( protons_multiRP_byArm_[ 0 ] ) > 0 )
#                 msk_protons &= np.array( ak.num( protons_multiRP_byArm_[ 1 ] ) > 0 )

#                 protons_multiRP_sel_ = protons_multiRP_[ msk_protons ]
#                 protons_singleRP_sel_ = protons_singleRP_[ msk_protons ]
#                 ppstracks_sel_ = ppstracks_[ msk_protons ]
#                 print ("\n")
#                 if debug:
#                     print ( msk_protons )
#                 print ( len( protons_multiRP_sel_ ) )
#                 print ( ak.num( protons_multiRP_sel_ ) )
#                 if debug:
#                     print ("\n")
#                     print ( ak.to_list( protons_multiRP_sel_ ) )
#                     print ("\n")
#                     print ( ak.to_list( protons_singleRP_sel_ ) )
#                     print ("\n")
#                     print ( ak.to_list( ppstracks_sel_ ) )

#                 counts_protons_ = len( protons_[ msk_protons ] )
#                 if not counts_label_protons_ in selections:
#                     selections = np.concatenate( ( selections, np.array( [ counts_label_protons_ ] ) ) )
#                     counts = np.concatenate( ( counts, np.array( [counts_protons_] ) ) )
#                 else:    
#                     counts[ selections == counts_label_protons_ ] += counts_protons_ 

#                 print ( selections )
#                 print ( counts )

#                 for col_ in columns_protons:
#                     protons_multiRP_list[ col_ ] = np.array( ak.flatten( protons_multiRP_sel_[ protons_keys[ col_ ] ] ) )

#                 arr_size_multiRP_ = len( protons_multiRP_list[ "xi" ] )
#                 print ( "Flattened array size multi-RP: {}".format( arr_size_multiRP_ ) )

#                 for col_ in columns_protons:
#                     protons_singleRP_list[ col_ ] = np.array( ak.flatten( protons_singleRP_sel_[ protons_keys[ col_ ] ] ) )

#                 arr_size_singleRP_ = len( protons_singleRP_list[ "xi" ] )
#                 print ( "Flattened array size single-RP: {}".format( arr_size_singleRP_ ) )

#                 for col_ in columns_ppstracks:
#                     ppstracks_list[ col_ ] = np.array( ak.flatten( ppstracks_sel_[ ppstracks_keys[ col_ ] ] ) )

#                 arr_size_ppstracks_ = len( ppstracks_list[ "x" ] )
#                 print ( "Flattened array size tracks: {}".format( arr_size_ppstracks_ ) )

#                 dset_multiRP_entries += arr_size_multiRP_
#                 dset_singleRP_entries += arr_size_singleRP_
#                 dset_ppstracks_entries += arr_size_ppstracks_

#                 if dset_multiRP_entries > dset_chunk_size:
#                     resize_factor_ = ( dset_multiRP_entries // dset_chunk_size )
#                     chunk_resize_  = resize_factor_ * dset_chunk_size

#                     print ( "Resizing output dataset by {} entries.".format( chunk_resize_ ) )
#                     dset_protons_multiRP.resize( ( dset_protons_multiRP.shape[0] + chunk_resize_ ), axis=0 )
#                     print ( "Dataset shape: {}".format( dset_protons_multiRP.shape ) )

#                     dset_multiRP_slice += resize_factor_
#                     # Count the rest to the chunk size 
#                     dset_multiRP_entries = ( dset_multiRP_entries % dset_chunk_size )

#                 if dset_singleRP_entries > dset_chunk_size:
#                     resize_factor_ = ( dset_singleRP_entries // dset_chunk_size )
#                     chunk_resize_  = resize_factor_ * dset_chunk_size

#                     print ( "Resizing output dataset by {} entries.".format( chunk_resize_ ) )
#                     dset_protons_singleRP.resize( ( dset_protons_singleRP.shape[0] + chunk_resize_ ), axis=0 )
#                     print ( "Dataset shape: {}".format( dset_protons_singleRP.shape ) )

#                     dset_singleRP_slice += resize_factor_
#                     # Count the rest to the chunk size 
#                     dset_singleRP_entries = ( dset_singleRP_entries % dset_chunk_size )

#                 if dset_ppstracks_entries > dset_chunk_size:
#                     resize_factor_ = ( dset_ppstracks_entries // dset_chunk_size )
#                     chunk_resize_  = resize_factor_ * dset_chunk_size

#                     print ( "Resizing output dataset by {} entries.".format( chunk_resize_ ) )
#                     dset_ppstracks.resize( ( dset_ppstracks.shape[0] + chunk_resize_ ), axis=0 )
#                     print ( "Dataset shape: {}".format( dset_ppstracks.shape ) )

#                     dset_ppstracks_slice += resize_factor_
#                     # Count the rest to the chunk size 
#                     dset_ppstracks_entries = ( dset_ppstracks_entries % dset_chunk_size )

#                 print ( "Stacking data." )
#                 data_protons_multiRP_ = np.stack( list( protons_multiRP_list.values() ), axis=1 )
#                 print ( data_protons_multiRP_.shape )
#                 print ( data_protons_multiRP_ )

#                 data_protons_singleRP_ = np.stack( list( protons_singleRP_list.values() ), axis=1 )
#                 print ( data_protons_singleRP_.shape )
#                 print ( data_protons_singleRP_ )

#                 data_ppstracks_ = np.stack( list( ppstracks_list.values() ), axis=1 )
#                 print ( data_ppstracks_.shape )
#                 print ( data_ppstracks_ )

#                 dset_idx_next_ = dset_multiRP_idx + arr_size_multiRP_
#                 print ( "Slice: {}".format( dset_multiRP_slice ) )
#                 print ( "Writing in positions ({},{})".format( dset_multiRP_idx, dset_idx_next_ ) )
#                 dset_protons_multiRP[ dset_multiRP_idx : dset_idx_next_ ] = data_protons_multiRP_
#                 dset_multiRP_idx = dset_idx_next_ 

#                 dset_idx_next_ = dset_singleRP_idx + arr_size_singleRP_
#                 print ( "Slice: {}".format( dset_singleRP_slice ) )
#                 print ( "Writing in positions ({},{})".format( dset_singleRP_idx, dset_idx_next_ ) )
#                 dset_protons_singleRP[ dset_singleRP_idx : dset_idx_next_ ] = data_protons_singleRP_
#                 dset_singleRP_idx = dset_idx_next_ 

#                 dset_idx_next_ = dset_ppstracks_idx + arr_size_ppstracks_
#                 print ( "Slice: {}".format( dset_ppstracks_slice ) )
#                 print ( "Writing in positions ({},{})".format( dset_ppstracks_idx, dset_idx_next_ ) )
#                 dset_ppstracks[ dset_ppstracks_idx : dset_idx_next_ ] = data_ppstracks_
#                 dset_ppstracks_idx = dset_idx_next_ 

#             # Iteration on input files
#             root_.close()

#         # Reduce dataset to its final size 
#         print ( "Reduce dataset." )
#         dset_protons_multiRP.resize( ( dset_multiRP_idx ), axis=0 ) 
#         print ( "Dataset shape: {}".format( dset_protons_multiRP.shape ) )

#         dset_protons_singleRP.resize( ( dset_singleRP_idx ), axis=0 ) 
#         print ( "Dataset shape: {}".format( dset_protons_singleRP.shape ) )

#         dset_ppstracks.resize( ( dset_ppstracks_idx ), axis=0 ) 
#         print ( "Dataset shape: {}".format( dset_ppstracks.shape ) )

#         print ( "Writing column names and event counts.")

#         columns_protons_ = np.array( columns_protons, dtype='S' )
#         print ( columns_protons_ )

#         columns_ppstracks_ = np.array( columns_ppstracks, dtype='S' )
#         print ( columns_ppstracks_ )

#         event_counts_ = counts
#         print ( event_counts_ )

#         selections_ = np.array( selections, dtype='S' )
#         print ( selections_ )

#         dset_columns_protons = f.create_dataset( 'columns_protons', data=columns_protons_ )
#         dset_columns_ppstracks = f.create_dataset( 'columns_ppstracks', data=columns_ppstracks_ )
#         dset_counts = f.create_dataset( 'event_counts', data=event_counts_ )
#         dset_selections = f.create_dataset( 'selections', data=selections_ )

#         print ( dset_protons_multiRP )
#         print ( dset_protons_multiRP[-1] )
#         print ( dset_protons_singleRP )
#         print ( dset_protons_singleRP[-1] )   
#         print ( dset_ppstracks )
#         print ( dset_ppstracks[-1] )   

#         print ( dset_columns_protons )
#         print ( list( dset_columns_protons ) )
#         print ( dset_columns_ppstracks )
#         print ( list( dset_columns_ppstracks ) )   
#         print ( dset_counts )
#         print ( list( dset_counts ) )
#         print ( dset_selections )
#         print ( list( dset_selections ) )

### Signal

In [ ]:
run_tables = True

debug = False

fileNames_A0W_ACW = {}
fileNames_A0W_ACW[ "A0W1e-6" ] = [
    "/eos/user/a/antoniov/Workspace/analysis/data/PPS/GGToWW_bSM-A0W1e-6_13TeV-fpmc-herwig6-signal-proton-propagation-dilepton/200420_160422/0000/SlimmedNtuple_merged_0.root"
]
fileNames_A0W_ACW[ "A0W2e-6" ] = [
    "/eos/user/a/antoniov/Workspace/analysis/data/PPS/GGToWW_bSM-A0W2e-6_13TeV-fpmc-herwig6-signal-proton-propagation-dilepton/200420_160449/0000/SlimmedNtuple_merged_0.root"
]
fileNames_A0W_ACW[ "A0W5e-6" ] = [
    "/eos/user/a/antoniov/Workspace/analysis/data/PPS/GGToWW_bSM-A0W5e-6_13TeV-fpmc-herwig6-signal-proton-propagation-dilepton/200420_160355/0000/SlimmedNtuple_merged_0.root"
]
fileNames_A0W_ACW[ "ACW5e-6" ] = [
    "/eos/user/a/antoniov/Workspace/analysis/data/PPS/GGToWW_bSM-ACW5e-6_13TeV-fpmc-herwig6-signal-proton-propagation-dilepton/200424_185019/0000/SlimmedNtuple_1.root"
]
fileNames_A0W_ACW[ "ACW8e-6" ] = [
    "/eos/user/a/antoniov/Workspace/analysis/data/PPS/GGToWW_bSM-ACW8e-6_13TeV-fpmc-herwig6-signal-proton-propagation-dilepton/200424_185049/0000/SlimmedNtuple_merged_0.root"
]
fileNames_A0W_ACW[ "ACW2e-5" ] = [
    "/eos/user/a/antoniov/Workspace/analysis/data/PPS/GGToWW_bSM-ACW2e-5_13TeV-fpmc-herwig6-signal-proton-propagation-dilepton/200424_185121/0000/SlimmedNtuple_merged_0.root"
]

#label = "GGToWW-AQGC-test"
label = "GGToWW-AQGC"
firstEvent = None
entryStop = None
#read_size = "150MB"
step_size = 100000

if run_tables:
    for key_ in fileNames_A0W_ACW:
        import time
        print( time.strftime("%Y/%m/%d %H:%M:%S", time.localtime() ) )
        time_s_ = time.time()
        
        print ( key_, fileNames_A0W_ACW[ key_ ] )
        label_ = "{}-{}".format( label, key_ )
        create_table( fileNames_A0W_ACW[ key_ ], label=label_, mix_protons=False, runOnMC=True, step_size=step_size, firstEvent=firstEvent, entryStop=entryStop, debug=debug )
        
        time_e_ = time.time()
        print ( "Total time elapsed: {:.0f}".format( time_e_ - time_s_ ) )

### Signal with mixed protons

In [ ]:
run_tables = True

debug = False

proton_files_ = [
    "/eos/home-a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017B-withDilepton/SlimmedNtuple_merged.root",
    "/eos/home-a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017C-withDilepton/SlimmedNtuple_merged.root",
    "/eos/home-a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017D-withDilepton/SlimmedNtuple_merged.root",
    "/eos/home-a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017E-withDilepton/SlimmedNtuple_merged.root",
    "/eos/home-a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017F-withDilepton/SlimmedNtuple_merged.root"
]

label = "GGToWW-AQGC-mix_protons"
firstEvent = None
entryStop = None
step_size = 100000

if run_tables:
    for key_ in fileNames_A0W_ACW:
        import time
        print( time.strftime("%Y/%m/%d %H:%M:%S", time.localtime() ) )
        time_s_ = time.time()
        
        print ( key_, fileNames_A0W_ACW[ key_ ] )
        label_ = "{}-{}".format( label, key_ )
        create_table( fileNames_A0W_ACW[ key_ ], label=label_, mix_protons=True, proton_files=proton_files_, runOnMC=True,
                                                 step_size=step_size, firstEvent=firstEvent, entryStop=entryStop, debug=debug )
        
        time_e_ = time.time()
        print ( "Total time elapsed: {:.0f}".format( time_e_ - time_s_ ) )

### Data

In [ ]:
run_tables = False

debug = False

fileNames_data = {}
fileNames_data[ "2017B" ] = [
#     "/eos/user/a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017B-Dilepton/190621_214052/0000/SlimmedNtuple_merged_0.root"
#     "/eos/user/a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017B-withDilepton/0000/SlimmedNtuple_merged_0.root"
    "/eos/home-a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017B-withDilepton/SlimmedNtuple_merged.root"
]
fileNames_data[ "2017C" ] = [
#     "/eos/user/a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017C-Dilepton/190621_214124/0000/SlimmedNtuple_merged_0.root",
#     "/eos/user/a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017C-Dilepton/190621_214124/0000/SlimmedNtuple_merged_1.root"
#     "/eos/user/a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017C-withDilepton/0000/SlimmedNtuple_merged_0.root",
#     "/eos/user/a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017C-withDilepton/0000/SlimmedNtuple_merged_1.root",
#     "/eos/user/a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017C-withDilepton/0001/SlimmedNtuple_merged_0.root"
    "/eos/home-a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017C-withDilepton/SlimmedNtuple_merged.root"
]
fileNames_data[ "2017D" ] = [
#     "/eos/user/a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017D-Dilepton/190621_214154/SlimmedNtuple_merged_0.root"
#     "/eos/user/a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017D-withDilepton/0000/SlimmedNtuple_merged_0.root",
#     "/eos/user/a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017D-withDilepton/0000/SlimmedNtuple_merged_1.root"
    "/eos/home-a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017D-withDilepton/SlimmedNtuple_merged.root"
]
fileNames_data[ "2017E" ] = [
#     "/eos/user/a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017E-Dilepton/190621_214224/0000/SlimmedNtuple_merged_0.root",
#     "/eos/user/a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017E-Dilepton/190621_214224/0000/SlimmedNtuple_merged_1.root"
#     "/eos/user/a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017E-withDilepton/0000/SlimmedNtuple_merged_0.root",
#     "/eos/user/a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017E-withDilepton/0000/SlimmedNtuple_merged_1.root",
#     "/eos/user/a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017E-withDilepton/0001/SlimmedNtuple_merged_0.root"
    "/eos/home-a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017E-withDilepton/SlimmedNtuple_merged.root"
]
fileNames_data[ "2017F" ] = [
#     "/eos/user/a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017F-Dilepton/190621_214253/0000/SlimmedNtuple_merged_0.root",
#     "/eos/user/a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017F-Dilepton/190621_214253/0000/SlimmedNtuple_merged_1.root"
#     "/eos/user/a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017F-Dilepton/190621_214253/0001/SlimmedNtuple_merged_0.root"
#     "/eos/user/a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017F-withDilepton/0000/SlimmedNtuple_merged_0.root",
#     "/eos/user/a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017F-withDilepton/0000/SlimmedNtuple_merged_1.root",
#     "/eos/user/a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017F-withDilepton/0001/SlimmedNtuple_merged_0.root",
#     "/eos/user/a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017F-withDilepton/0001/SlimmedNtuple_merged_1.root",
#     "/eos/user/a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017F-withDilepton/0002/SlimmedNtuple_merged_0.root",
#     "/eos/user/a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017F-withDilepton/0003/SlimmedNtuple_merged_0.root"
    "/eos/home-a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017F-withDilepton/SlimmedNtuple_merged.root"
]

#label = "data-test"
#label = "data-test-rnd-run-info"
label = "data"
firstEvent = None
entryStop = None
#read_size = "150MB"
step_size = 100000

if run_tables:
    for key_ in fileNames_data:
        import time
        print( time.strftime("%Y/%m/%d %H:%M:%S", time.localtime() ) )
        time_s_ = time.time()

        print ( key_, fileNames_data[ key_ ] )
        label_ = "{}-{}".format( label, key_ )
        create_table( fileNames_data[ key_ ], label=label_, random_protons=False, step_size=step_size, firstEvent=firstEvent, entryStop=entryStop, debug=debug )
        
        time_e_ = time.time()
        print ( "Total time elapsed: {:.0f}".format( time_e_ - time_s_ ) )

### Data with random protons

In [ ]:
run_tables = True

debug = False

#label = "data-random-resample_20-test"
#label = "data-random-resample_20-test-rnd-run-info"
label = "data-random-resample_20"
firstEvent = None
entryStop = None
#read_size = "150MB"
step_size = 50000
resample_factor = 20

if run_tables:
    for key_ in fileNames_data:
        import time
        print( time.strftime("%Y/%m/%d %H:%M:%S", time.localtime() ) )
        time_s_ = time.time()

        print ( key_, fileNames_data[ key_ ] )
        label_ = "{}-{}".format( label, key_ )
        create_table( fileNames_data[ key_ ], label=label_, random_protons=True, resample_factor=resample_factor, step_size=step_size, firstEvent=firstEvent, entryStop=entryStop, debug=debug )
        
        time_e_ = time.time()
        print ( "Total time elapsed: {:.0f}".format( time_e_ - time_s_ ) )

In [ ]:
def get_data( fileNames ):
    df_protons_multiRP_list = []
    df_protons_singleRP_list = []
    df_ppstracks_list = []
    df_counts_list = []

    for file_ in fileNames:
        print ( file_ )
        with h5py.File( file_, 'r' ) as f:
            print ( list(f.keys()) )

            dset_protons_multiRP = f['protons_multiRP']
            print ( dset_protons_multiRP.shape )
            print ( dset_protons_multiRP[:,:] )

            dset_protons_singleRP = f['protons_singleRP']
            print ( dset_protons_singleRP.shape )
            print ( dset_protons_singleRP[:,:] )

            dset_ppstracks = f['ppstracks']
            print ( dset_ppstracks.shape )
            print ( dset_ppstracks[:,:] )

            dset_columns_protons_multiRP = f['columns_protons_multiRP']
            print ( dset_columns_protons_multiRP.shape )
            columns_protons_multiRP = [ item.decode("utf-8") for item in list( dset_columns_protons_multiRP ) ]
            print ( columns_protons_multiRP )

            dset_columns_protons_singleRP = f['columns_protons_singleRP']
            print ( dset_columns_protons_singleRP.shape )
            columns_protons_singleRP = [ item.decode("utf-8") for item in list( dset_columns_protons_singleRP ) ]
            print ( columns_protons_singleRP )
            
            dset_columns_ppstracks = f['columns_ppstracks']
            print ( dset_columns_ppstracks.shape )
            columns_ppstracks = [ item.decode("utf-8") for item in list( dset_columns_ppstracks ) ]
            print ( columns_ppstracks )

            dset_selections = f['selections']
            selections_ = [ item.decode("utf-8") for item in dset_selections ]
            print ( selections_ )

            dset_counts = f['event_counts']
            df_counts_list.append( pd.Series( dset_counts, index=selections_ ) )
            print ( df_counts_list[-1] )

            chunk_size = 1000000
            entries_protons_multiRP = dset_protons_multiRP.shape[0]
            start_ = list( range( 0, entries_protons_multiRP, chunk_size ) )
            stop_ = start_[1:]
            stop_.append( entries_protons_multiRP )
            print ( start_ )
            print ( stop_ )
            astype_dict_protons_ = {
                "run": "int64", "lumiblock": "int64", "event": "int64", "slice": "int32",
                "ismultirp": "int32", "rpid": "int32", "arm": "int32", "random": "int32",
                "muon0_charge": "int32",
                "nVertices": "int32",
                "num_bjets_ak8": "int32", "num_bjets_ak4": "int32", "num_jets_ak4": "int32",
                "pfcand_nextracks": "int32", "pfcand_nextracks_noDRl": "int32",
                "trackpixshift1": "int32", "rpid1": "int32"
                }
            astype_dict_multiRP_ = astype_dict_protons_.copy()

            if "rpid2" in columns_protons_multiRP:
                astype_dict_multiRP_.update( { "trackpixshift2": "int32", "rpid2": "int32" } )

            if "run_mc" in columns_protons_multiRP:
                astype_dict_multiRP_.update( { "run_mc": "int64" } )

            if "run_rnd" in columns_protons_multiRP:
                # astype_dict_multiRP_.update( { "run_rnd": "int64", "lumiblock_rnd": "int64", "event_rnd": "int64", "slice_rnd": "int32" } )
                astype_dict_multiRP_.update( { "run_rnd": "int64", "lumiblock_rnd": "int64", "event_rnd": "int64" } )
                
            for idx in range( len( start_ ) ):
                print ( start_[idx], stop_[idx] )
                #print ( dset[ start_[idx] : stop_[idx] ] )
                df_ = pd.DataFrame( dset_protons_multiRP[ start_[idx] : stop_[idx] ], columns=columns_protons_multiRP ).astype( astype_dict_multiRP_ )
                df_protons_multiRP_list.append( df_ )
                print ( df_protons_multiRP_list[-1].head() )
                print ( "Data set size: {}".format( len( df_protons_multiRP_list[-1] ) ) )

            entries_protons_singleRP = dset_protons_singleRP.shape[0]
            start_ = list( range( 0, entries_protons_singleRP, chunk_size ) )
            stop_ = start_[1:]
            stop_.append( entries_protons_singleRP )
            print ( start_ )
            print ( stop_ )
            astype_dict_singleRP_ = astype_dict_protons_.copy()

            if "run_mc" in columns_protons_singleRP:
                astype_dict_singleRP_.update( { "run_mc": "int64" } )

            if "run_rnd" in columns_protons_singleRP:
                # astype_dict_singleRP_.update( { "run_rnd": "int64", "lumiblock_rnd": "int64", "event_rnd": "int64", "slice_rnd": "int32" } )
                astype_dict_singleRP_.update( { "run_rnd": "int64", "lumiblock_rnd": "int64", "event_rnd": "int64" } )
                
            for idx in range( len( start_ ) ):
                print ( start_[idx], stop_[idx] )
                #print ( dset[ start_[idx] : stop_[idx] ] )
                df_ = pd.DataFrame( dset_protons_singleRP[ start_[idx] : stop_[idx] ], columns=columns_protons_singleRP ).astype( astype_dict_singleRP_ )
                df_protons_singleRP_list.append( df_ )
                print ( df_protons_singleRP_list[-1].head() )
                print ( "Data set size: {}".format( len( df_protons_singleRP_list[-1] ) ) )

            entries_ppstracks = dset_ppstracks.shape[0]
            start_ = list( range( 0, entries_ppstracks, chunk_size ) )
            stop_ = start_[1:]
            stop_.append( entries_ppstracks )
            print ( start_ )
            print ( stop_ )
            astype_dict_ppstracks_ = { "run": "int64", "lumiblock": "int64", "event": "int64", "slice": "int32", "rpid": "int32" }

            if "run_mc" in columns_ppstracks:
                astype_dict_ppstracks_.update( { "run_mc": "int64" } )

            if "run_rnd" in columns_ppstracks:
                # astype_dict_ppstracks_.update( { "run_rnd": "int64", "lumiblock_rnd": "int64", "event_rnd": "int64", "slice_rnd": "int32" } )
                astype_dict_ppstracks_.update( { "run_rnd": "int64", "lumiblock_rnd": "int64", "event_rnd": "int64" } )
                
            for idx in range( len( start_ ) ):
                print ( start_[idx], stop_[idx] )
                #print ( dset[ start_[idx] : stop_[idx] ] )
                df_ = pd.DataFrame( dset_ppstracks[ start_[idx] : stop_[idx] ], columns=columns_ppstracks ).astype( astype_dict_ppstracks_ )
                df_ppstracks_list.append( df_ )
                print ( df_ppstracks_list[-1].head() )
                print ( "Data set size: {}".format( len( df_ppstracks_list[-1] ) ) )

    df_counts = df_counts_list[0]
    for idx in range( 1, len( df_counts_list ) ):
        df_counts = df_counts.add( df_counts_list[idx] )
    print ( df_counts )

    df_protons_multiRP = pd.concat( df_protons_multiRP_list )
    print (df_protons_multiRP)

    df_protons_singleRP = pd.concat( df_protons_singleRP_list )
    print (df_protons_singleRP)

    df_ppstracks = pd.concat( df_ppstracks_list )
    print (df_ppstracks)
    
    return ( df_counts, df_protons_multiRP, df_protons_singleRP, df_ppstracks )
    

In [ ]:
# fileNames_data = [
#     'output-data-2017B.h5',
#     'output-data-2017C.h5',
#     'output-data-2017D.h5',
#     'output-data-2017E.h5',
#     'output-data-2017F.h5'
# ]

# fileNames_data = [
#     'output-data-random-resample_20-2017B.h5'
# ]

# df_counts, df_protons_multiRP, df_protons_singleRP, df_ppstracks = get_data( fileNames_data )

fileNames = [
#     'output-GGToWW-AQGC-A0W1e-6.h5'
    'output-GGToWW-AQGC-mix_protons-A0W2e-6.h5'
]

df_counts, df_protons_multiRP, df_protons_singleRP, df_ppstracks = get_data( fileNames )

In [ ]:
df_counts

In [ ]:
# run_str_ = "run"
run_str_ = "run_mc"

from processing import df_run_ranges

df_protons_multiRP.loc[ :, "period" ] = np.nan
for idx_ in range( df_run_ranges.shape[0] ):
    msk_period_ = ( ( df_protons_multiRP.loc[ :, run_str_ ] >= df_run_ranges.iloc[ idx_ ][ "min" ] ) & ( df_protons_multiRP.loc[ :, run_str_ ] <= df_run_ranges.iloc[ idx_ ][ "max" ] ) )
    sum_period_ = np.sum( msk_period_ )
    if sum_period_ > 0:
        period_key_ = df_run_ranges.index[ idx_ ]
        df_protons_multiRP.loc[ :, "period" ].loc[ msk_period_ ] = period_key_
        print ( "{}: {}".format( period_key_, sum_period_ ) )
print ( df_protons_multiRP.loc[ :, "period" ] )

In [ ]:
# df_protons_multiRP[ [ 'run', 'lumiblock', 'event', 'slice', 'crossingAngle', 'betaStar' ] ]
# df_protons_multiRP[ [ 'run', 'period', 'lumiblock', 'event', 'slice', 'crossingAngle', 'betaStar' ] ]
# df_protons_multiRP[ [ 'run', 'lumiblock', 'event', 'slice', 'crossingAngle', 'run_rnd', 'lumiblock_rnd', 'event_rnd', 'crossingAngle_rnd' ] ]
# df_protons_multiRP[ [ 'run', 'lumiblock', 'event', 'slice', 'crossingAngle', 'run_mc' ] ]
df_protons_multiRP[ [ 'run', 'lumiblock', 'event', 'slice', 'crossingAngle', 'betaStar', 'run_mc', 'period', 'run_rnd', 'lumiblock_rnd', 'event_rnd', 'crossingAngle_rnd', 'betaStar_rnd' ] ]

In [ ]:
df_protons_multiRP_index = df_protons_multiRP.set_index( ['run', 'lumiblock', 'event', 'slice'] )
df_protons_multiRP_index

In [ ]:
fig, axes = plt.subplots( 1, 2, figsize=(24,10) )
df_protons_multiRP[ df_protons_multiRP.arm == 0 ].hist( "xi", bins=10, range=(0.,0.2), ax=axes[0] )
df_protons_multiRP[ df_protons_multiRP.arm == 1 ].hist( "xi", bins=10, range=(0.,0.2), ax=axes[1] )

In [ ]:
df_protons_multiRP_events = df_protons_multiRP_index.drop( columns=[ 'xi', 'thx', 'thy', 't', 'ismultirp', 'rpid', 'arm', 'random', 'trackx1', 'tracky1', 'trackpixshift1', 'rpid1', 'trackx2', 'tracky2', 'trackpixshift2', 'rpid2' ] )
df_protons_multiRP_events = df_protons_multiRP_events[ ~df_protons_multiRP_events.index.duplicated(keep='first') ]
df_protons_multiRP_events

In [ ]:
# data_periods = [ "2017B", "2017C1", "2017C2", "2017D", "2017E", "2017F1", "2017F2", "2017F3" ]
# data_periods = [ "2017B" ]
# data_periods = [ "2017C1", "2017C2" ]
# data_periods = [ "2017D" ]
# data_periods = [ "2017E" ]
data_periods = [ "2017F1", "2017F2", "2017F3" ]
fig, axes = plt.subplots( 1, 2, figsize=(24,10) )
for period_ in data_periods:
    msk__ = df_protons_multiRP_events.loc[ :, "period" ] == period_
    df_protons_multiRP_events.loc[ msk__ ].hist( "crossingAngle", bins=40, range=(100.,180.), histtype='step', linewidth=2 , label=period_, ax=axes[0] )
    df_protons_multiRP_events.loc[ msk__ ].hist( "betaStar", bins=10, range=(0.,1.), histtype='step', linewidth=2, label=period_, ax=axes[1] )
axes[0].legend( loc='best', fontsize=16  )
axes[1].legend( loc='best', fontsize=16 )

In [ ]:
# data_periods = [ "2017B", "2017C1", "2017C2", "2017D", "2017E", "2017F1", "2017F2", "2017F3" ]
# data_periods = [ "2017B" ]
data_periods = [ "2017C1", "2017C2" ]
# data_periods = [ "2017D" ]
# data_periods = [ "2017E" ]
# data_periods = [ "2017F1", "2017F2", "2017F3" ]
fig, axes = plt.subplots( 1, 2, figsize=(24,10) )
for period_ in data_periods:
    msk__ = df_protons_multiRP_events.loc[ :, "period" ] == period_
    df_protons_multiRP_events.loc[ msk__ ].hist( "crossingAngle", bins=40, range=(100.,180.), histtype='step', linewidth=2 , label=period_, ax=axes[0] )
    df_protons_multiRP_events.loc[ msk__ ].hist( "crossingAngle_rnd", bins=40, range=(100.,180.), histtype='step', linewidth=2 , label=(period_ + " random"), ax=axes[0] )
    df_protons_multiRP_events.loc[ msk__ ].hist( "betaStar", bins=10, range=(0.,1.), histtype='step', linewidth=2, label=period_, ax=axes[1] )
    df_protons_multiRP_events.loc[ msk__ ].hist( "betaStar_rnd", bins=10, range=(0.,1.), histtype='step', linewidth=2, label=(period_ + " random"), ax=axes[1] )
axes[0].legend( loc='best', fontsize=16  )
axes[1].legend( loc='best', fontsize=16 )

In [ ]:
# data_periods = [ "2017B", "2017C1", "2017C2", "2017D", "2017E", "2017F1", "2017F2", "2017F3" ]
# data_periods = [ "2017B" ]
# data_periods = [ "2017C1", "2017C2" ]
# data_periods = [ "2017D" ]
# data_periods = [ "2017E" ]
data_periods = [ "2017F1", "2017F2", "2017F3" ]
fig, axes = plt.subplots( 1, 2, figsize=(24,10) )
colors_ = ('darkblue', 'lightcoral', 'forestgreen', 'purple', 'darkorange', 'silver', 'darkred', 'violet' )
for idx_, period_ in enumerate( data_periods ):
    msk__ = df_protons_multiRP_events.loc[ :, "period" ] == period_
    df_protons_multiRP_events.loc[ msk__ ].plot( "crossingAngle", "crossingAngle_rnd", kind='scatter', color=colors_[ idx_ ], label=period_, ax=axes[0] )
    df_protons_multiRP_events.loc[ msk__ ].plot( "betaStar", "betaStar_rnd", kind='scatter', color=colors_[ idx_ ], label=period_, ax=axes[1] )
axes[0].legend( loc='best', fontsize=16  )
axes[1].legend( loc='best', fontsize=16 )

In [ ]:
#list( df_protons_multiRP_index.groupby( ["run","lumiblock","event","slice"] ) )
#df_protons_multiRP_index.groupby( ["run","lumiblock","event","slice"] ).apply( lambda df_: print( df_.iloc[0].xi, df_.iloc[1].xi ) )
df_protons_multiRP_events_groupby = df_protons_multiRP_index[ ["xi"] ].groupby( ["run","lumiblock","event","slice"] )
df_protons_multiRP_events[ "MX" ] = df_protons_multiRP_events_groupby.apply( lambda df_: 13000. * np.sqrt( df_.iloc[0].xi * df_.iloc[1].xi ) )
df_protons_multiRP_events[ "YX" ] = df_protons_multiRP_events_groupby.apply( lambda df_: 0.5 * np.log( df_.iloc[0].xi / df_.iloc[1].xi ) )
df_protons_multiRP_events[ "diffMWW_MX" ]  = df_protons_multiRP_events[ "recoMWW" ] - df_protons_multiRP_events[ "MX" ]
df_protons_multiRP_events[ "ratioMWW_MX" ] = df_protons_multiRP_events[ "recoMWW" ] / df_protons_multiRP_events[ "MX" ]
df_protons_multiRP_events[ "diffYWW_YX" ]  = df_protons_multiRP_events[ "recoRapidityWW" ] - df_protons_multiRP_events[ "YX" ]
df_protons_multiRP_events

In [ ]:
fig, axes = plt.subplots( 1, 2, figsize=(24,10) )
df_protons_multiRP_events.hist( "MX", bins=50, range=(200.,2000.), ax=axes[0] )
df_protons_multiRP_events.hist( "YX", bins=50, range=(-3.,3.), ax=axes[1] )

In [ ]:
fig, axes = plt.subplots( 1, 2, figsize=(24,10) )
df_protons_multiRP_events.hist( "diffMWW_MX", bins=50, range=(-1000.,1000.), ax=axes[0] )
df_protons_multiRP_events.hist( "diffYWW_YX", bins=50, range=(-3.,3.), ax=axes[1] )

In [ ]:
df_protons_multiRP_events.hist( "ratioMWW_MX", bins=50, range=(0.,1.5), figsize=(12,10) )

In [ ]:
fig = plt.figure( figsize=(12,10) )
plt.plot( (0.,2.), (0.,0.), color='lightblue' )
plt.plot( (1.,1.), (-3.,3.), color='lightblue' )
ax = plt.gca()
df_protons_multiRP_events.plot( "ratioMWW_MX", "diffYWW_YX", 'scatter', ax=ax )
plt.xlim(0., 2.0)
plt.ylim(-3., 3.)

In [ ]:
df_protons_singleRP

In [ ]:
df_protons_singleRP_index = df_protons_singleRP.set_index( ['run', 'lumiblock', 'event', 'slice'] )
df_protons_singleRP_index

In [ ]:
df_ppstracks

In [ ]:
df_ppstracks_index = df_ppstracks.set_index( ['run', 'lumiblock', 'event', 'slice'] )
#df_ppstracks_index
df_ppstracks_index[ ( df_ppstracks_index.rpid != 16 ) & ( df_ppstracks_index.rpid != 116 ) ]

In [ ]:
fig, axes = plt.subplots( 1, 2, figsize=(24,10) )
df_ppstracks[ df_ppstracks.rpid == 23 ].hist( "x", bins=40, range=(0.,80.), ax=axes[0] )
df_ppstracks[ df_ppstracks.rpid == 23 ].hist( "y", bins=20, range=(-15.,15.), ax=axes[1] )

### References

In [ ]:
np.info( ak.concatenate )

In [ ]:
np.info( pd.DataFrame.hist )

In [ ]:
np.info( pd.DataFrame.plot.scatter )

In [ ]:
np.info( plt.plot )